In [12]:
library(infercnv)

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 



In [2]:
library(rtracklayer)

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb



In [3]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::%within%() masks IRanges::%within%()
✖ dplyr::collapse()     masks IRanges::collapse()
✖ dplyr::combine()      masks BiocGenerics::combine()
✖ dplyr::desc()         masks IRanges::desc()
✖ tidyr::expand()       masks S4Vectors::expand()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::first()        masks S4Vectors::first()
✖ dplyr::lag()          masks stats::lag()
✖ ggplot2::Position()   masks BiocGenerics::Position(), base::Position()
✖ purrr::reduce()       masks GenomicRanges::reduce(), IRanges::reduce()
✖ dplyr::rename()       masks S4Vectors::rename()
✖ lubridate::second()   masks S4Vectors::second()
✖ lubridate::second<-

# Prepare files for infercnv

### prepare gene position file (one time only, then read from disk)

#### I downloaded the gtf file from https://support.10xgenomics.com/single-cell-gene-expression/software/release-notes/build#grch38_1.2.0 and filtered as per the instructions there

In [4]:
gtf <- rtracklayer::import('data//Homo_sapiens.GRCh38.84.filtered.gtf')

In [5]:
gtf_df=as.data.frame(gtf)

In [6]:
head(gtf_df)

,seqnames,start,end,width,strand,source,type,score,phase,gene_id,⋯,havana_transcript,havana_transcript_version,tag,transcript_support_level,exon_number,exon_id,exon_version,ccds_id,protein_id,protein_version
,<fct>,<int>,<int>,<int>,<fct>,<fct>,<fct>,<dbl>,<int>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,29554,31109,1556,+,havana,gene,NA,NA,ENSG00000243485,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,1,29554,31097,1544,+,havana,transcript,NA,NA,ENSG00000243485,⋯,OTTHUMT00000002840,1,basic,5,NA,NA,NA,NA,NA,NA
3,1,29554,30039,486,+,havana,exon,NA,NA,ENSG00000243485,⋯,OTTHUMT00000002840,1,basic,5,1,ENSE00001947070,1,NA,NA,NA
4,1,30564,30667,104,+,havana,exon,NA,NA,ENSG00000243485,⋯,OTTHUMT00000002840,1,basic,5,2,ENSE00001922571,1,NA,NA,NA
5,1,30976,31097,122,+,havana,exon,NA,NA,ENSG00000243485,⋯,OTTHUMT00000002840,1,basic,5,3,ENSE00001827679,1,NA,NA,NA
6,1,30267,31109,843,+,havana,transcript,NA,NA,ENSG00000243485,⋯,OTTHUMT00000002841,2,basic,5,NA,NA,NA,NA,NA,NA


#### drop duplicate rows -- they're really similar so can just randomly pick one

In [7]:
length(unique((gtf_df %>% filter(type=="gene") %>% select(gene_name, seqnames, start, end))$gene_name))

[1] 33660

In [8]:
nrow(gtf_df %>% filter(type=="gene") %>% select(start, end, source, gene_id, gene_name))

[1] 33694

In [9]:
gtf_df = subset(gtf_df %>% filter(type=="gene") %>% select(gene_name, seqnames, start, end),!duplicated((gtf_df %>% filter(type=="gene"))$gene_name))
nrow(gtf_df)


[1] 33660

#### filter to genes on the 23 chromosomes

In [10]:
gtf_df = gtf_df %>% filter(seqnames %in% c(c(1:22), "X", "Y"))

In [52]:
write.table(gtf_df, file="data/infercnv_input_files/cd138_mm/gene_pos.txt", col.names=F, row.names=F, quote=F, sep="\t")

#### prepared the other 2 input files in python, writing from anndata object to disk

# Run infercnv

#### start by running with analysis_mode="subclusters" through step 15 and inspect outputs -- it finds too many subclusters, going to run with analysis_mode="samples" (below)

In [13]:
infercnv_obj = CreateInfercnvObject(raw_counts_matrix="data/infercnv_input_files/crc/GSE132465_GEO_processed_CRC_10X_raw_UMI_count_matrix.txt",
                                    annotations_file="data/infercnv_input_files/crc/cell_annots.txt",
                                    delim="\t",
                                    gene_order_file="data/infercnv_input_files/cd138_mm/gene_pos.txt",
                                    ref_group_names=c('T cells','B cells','Stromal cells','Epithelial cells','Myeloids','Mast cells'))

INFO [2023-08-08 14:25:32] Parsing matrix: data/infercnv_input_files/crc/GSE132465_GEO_processed_CRC_10X_raw_UMI_count_matrix.txt
INFO [2023-08-08 14:39:02] Parsing gene order file: data/infercnv_input_files/cd138_mm/gene_pos.txt
INFO [2023-08-08 14:39:02] Parsing cell annotations file: data/infercnv_input_files/crc/cell_annots.txt
INFO [2023-08-08 14:39:02] ::order_reduce:Start.
INFO [2023-08-08 14:39:12] .order_reduce(): expr and order match.
INFO [2023-08-08 14:39:26] ::process_data:order_reduce:Reduction from positional data, new dimensions (r,c) = 33694,63689 Total=622669375 Min=0 Max=26489.
INFO [2023-08-08 14:39:35] num genes removed taking into account provided gene ordering list: 76 = 0.225559446785778% removed.
INFO [2023-08-08 14:39:37] -filtering out cells < 100 or > Inf, removing 0 % of cells
WARN [2023-08-08 14:39:57] Please use "options(scipen = 100)" before running infercnv if you are using the analysis_mode="subclusters" option or you may encounter an error while the h

In [15]:
options(scipen = 100)

In [16]:
# perform infercnv operations to reveal cnv signal
infercnv_obj = infercnv::run(infercnv_obj,
                             cutoff=0.1,  # use 1 for smart-seq, 0.1 for 10x-genomics
                             out_dir="outputs/infercnv/crc/",  # dir is auto-created for storing outputs
                             cluster_by_groups=T,   # cluster
                             denoise=T,
                             HMM=T,
                            analysis_mode='subclusters',
                                       leiden_resolution=0.01, #initial run returned too many subclusters
                             )

INFO [2023-08-08 14:45:18] ::process_data:Start
INFO [2023-08-08 14:45:18] Checking for saved results.
INFO [2023-08-08 14:45:18] Trying to reload from step 1
INFO [2023-08-08 14:46:21] Using backup from step 1
INFO [2023-08-08 14:46:21] 

	STEP 1: incoming data

INFO [2023-08-08 14:46:21] 

	STEP 02: Removing lowly expressed genes

INFO [2023-08-08 14:46:21] ::above_min_mean_expr_cutoff:Start
INFO [2023-08-08 14:46:26] Removing 26541 genes from matrix as below mean expr threshold: 0.1
INFO [2023-08-08 14:46:29] validating infercnv_obj
INFO [2023-08-08 14:46:29] There are 7077 genes and 63689 cells remaining in the expr matrix.
INFO [2023-08-08 14:46:49] no genes removed due to min cells/gene filter
INFO [2023-08-08 14:48:52] 

	STEP 03: normalization by sequencing depth

INFO [2023-08-08 14:48:52] normalizing counts matrix by depth
INFO [2023-08-08 14:49:03] Computed total sum normalization factor as median libsize: 3896.000000
INFO [2023-08-08 14:49:05] Adding h-spike
INFO [2023-08-0

Centering and scaling data matrix

PC_ 1 
Positive:  GPN2, SFN, ARID1A, RPS6KA1, NUDC, CD52, SLC9A1, SH3BGRL3, TMEM57, TMEM50A 
	   LDLRAP1, RSRP1, MTFR1L, SEPN1, ZNF593, HMGN2, STMN1, TMEM222, SYF2, SYTL1 
	   RUNX3, WASF2, CLIC4, IFI6, SRRM1, STX12, RCAN3, SRSF10, PPP1R8, PNRC2 
Negative:  PARD6B, PTPN1, CEBPB, TMEM189, SNAI1, RNF114, SPATA2, B4GALT5, ZFAS1, YTHDF1 
	   ZNFX1, DDX27, ARFGAP1, STAU1, PPDPF, CSE1L, PTK6, ARFGEF2, HELZ2, SULF2 
	   KIF3B, STMN3, NCOA3, POFUT1, ARFRP1, ZMYND8, PLAGL2, SLC2A4RG, TM9SF4, TP53RK 
PC_ 2 
Positive:  SPP1, PPM1K, SPARCL1, PYURF, NUDT9, FAM13A, HSD17B11, GPRIN3, CDS1, PDLIM5 
	   MRPS18C, COQ2, RAP1GDS1, PLAC8, TSPAN5, COPS4, EIF4E, THAP9-AS1, METAP1, SEC31A 
	   ADH5, ENOPH1, ADH1B, HNRNPDL, ADH1C, HNRNPD, DAPP1, RASGEF1B, ANTXR2, LAMTOR3 
Negative:  OVOL1, RNASEH2C, CFL1, MUS81, KAT5, RELA, MAP3K11, SIPA1, EHBP1L1, FAM89B 
	   EIF1AD, SSSCA1, BANF1, LTBP3, MRPL49, CAPN1, SYVN1, SF3B2, FAU, CDC42EP2 
	   SCYL1, TM7SF2, DPF2, RAB1B, FRMD8, VPS5

INFO [2023-08-08 16:02:25] define_signif_tumor_subclusters(), tumor: malignant_SMC02


Centering and scaling data matrix

PC_ 1 
Positive:  ADAMDEC1, ADAM28, PPP2R2A, SLC25A37, R3HCC1, BNIP3L, TNFRSF10B, DPYSL2, EGR3, PTK2B 
	   BIN3, PDLIM2, EPHX2, SORBS3, CLU, PPP3CC, CCDC25, SLC39A14, REEP4, DUSP4 
	   DOK2, SARAF, ATP6V1B2, RP11-489E7.4, INTS10, LEPROTL1, RP11-51J9.5, ASAH1, DCTN6, RBPMS 
Negative:  KIF3B, NOL4L, ASXL1, COMMD7, CBFA2T2, CDK5RAP1, POFUT1, MAPRE1, NECAB3, CHMP4B 
	   RALY, PLAGL2, EIF2S2, TM9SF4, AHCY, PDRG1, DSTN, SNRPB2, KIF16B, TPX2 
	   ITCH, RRBP1, NDUFAF5, ESF1, JAG1, SNX5, BCL2L1, MKKS, DYNLRB1, PLCB4 
PC_ 2 
Positive:  APOBEC3G, RPL3, APOBEC3C, CBX6, MIEF1, SUN2, ATF4, GTPBP1, JOSD1, RPS19BP1 
	   SNX6, BAZ1A, EAPP, SRP54, SPTSSA, FAM177A1, TOMM22, EGLN3, PPP2R3C, ARHGAP5 
	   HECTD1, PSMA6, STRN3, SCFD1, STXBP6, GZMB, NFKBIA, CBY1, RALGAPA1, GZMH 
Negative:  EREG, MTHFD2L, AREG, CXCL2, PARM1, CXCL3, RCHY1, G3BP2, USO1, CXCL5 
	   NAAA, PPBP, SDAD1, CXCL1, CXCL8, CXCL9, CXCL10, RASSF6, NUP54, ANKRD17 
	   SCARB2, DCK, SHROOM3, GRSF1, SEPT11, UT

INFO [2023-08-08 16:02:31] define_signif_tumor_subclusters(), tumor: malignant_SMC03


...

INFO [2023-08-08 16:05:45] define_signif_tumor_subclusters(), tumor: Myeloids


Centering and scaling data matrix

PC_ 1 
Positive:  CORO1B, RPS6KB2, TMEM134, TBC1D10C, AIP, CDK2AP2, PPP1CA, GSTP1, POLD4, RAD9A 
	   ANKRD13D, RCE1, RBM4, NDUFV1, LRFN4, RBM14, RHOD, ADRBK1, KDM2A, CCS 
	   NUDT8, CTSF, ZDHHC24, DPP3, UNC93B1, MRPL11, NDUFS8, BRMS1, CD248, YIF1A 
Negative:  AP2B1, TAF15, RP11-1094M14.11, CCL5, NLE1, CCL15, RFFL, CCL18, ZNF830, CCL13 
	   CCL3, CCL8, CCL11, CCL4, CCL2, TMEM98, MYO1D, CCL3L3, PSMD11, CCL4L2 
	   ZNF207, ZNHIT3, RHOT1, MYO19, SUZ12, GGNBP2, UTP6, DHRS11, COPRS, AATF 
PC_ 2 
Positive:  THAP9-AS1, COPS4, PLAC8, SEC31A, ENOPH1, COQ2, HNRNPDL, ANXA3, HNRNPD, MRPL1 
	   CNOT6L, ANTXR2, RASGEF1B, CXCL13, CCNG2, MRPS18C, CCNI, SEPT11, SHROOM3, CDS1 
	   SCARB2, NUP54, CXCL10, CXCL9, HSD17B11, SDAD1, NAAA, USO1, NUDT9, G3BP2 
Negative:  CTD-2336O2.1, AGPAT5, CLN8, FBXO25, DEFA6, ERICH1, DEFA5, CLDN23, MFHAS1, MSRA 
	   FDFT1, CTSB, CNOT7, SDS, SDSL, RITA1, TBX3, DDX54, MED13L, C12orf49 
	   OAS3, PAXIP1-AS1, LINC01003, TESC, INSIG1, KMT2C, GAL

INFO [2023-08-08 16:05:47] define_signif_tumor_subclusters(), tumor: Mast cells


Centering and scaling data matrix

PC_ 1 
Positive:  COX6C, POLR2K, STK3, HRSP12, RPL30, ENY2, MATN2, EBAG9, EIF3H, ENPP2 
	   MAL2, MRPL13, MED30, UTP23, RAD21, LAPTM4B, DERL1, C8orf76, MTDH, ZHX1 
	   ATAD2, CPQ, WDYHV1, SDC2, FBXO32, PTDSS1, UQCRB, ANXA13, PLEKHF2, NDUFAF6 
Negative:  ADNP, BCAS4, PARD6B, DPM1, NFATC2, ATP9A, PTPN1, TSHZ2, ZNF217, CEBPB 
	   PFDN4, TMEM189, FAM210B, AURKA, SNAI1, GABARAP, PHF23, ACADVL, XAF1, CTDNEP1 
	   RTFDC1, RNF114, ELP5, MED31, CLDN7, RAE1, SPATA2, EIF5A, TXNDC17, B4GALT5 
PC_ 2 
Positive:  U2SURP, ATP1B3, PLOD2, RNF7, PLSCR1, RASA2, GYG1, ZBTB38, TM4SF1, SLC25A36 
	   RBP1, TM4SF4, COPB2, MRPS22, WWTR1, COMMD2, ANAPC13, RNF13, AMOTL2, PFN2 
	   RYK, TSC22D2, SRPRB, SERP1, EIF2A, SELT, SIAH2, GPR171, CDV3, MBNL1 
Negative:  TRPM4, CD37, SNRNP70, PIH1D1, RUVBL2, BAX, FTL, NUCB1, PPP1R15A, BCAT2 
	   FUT2, RPL18, SULT2B1, LMTK3, CYTH2, GRWD1, KDELR1, EMP3, CARD8, SEPW1 
	   NPRL2, CYB561D2, RASSF1, TMEM115, GLTSCR2, RRP9, TUSC2, MAPKAPK3, PCBP4,

INFO [2023-08-08 16:05:48] -mirroring for hspike
INFO [2023-08-08 16:05:48] define_signif_tumor_subclusters(p_val=0.1
INFO [2023-08-08 16:05:48] define_signif_tumor_subclusters(), tumor: spike_tumor_cell_T cells
INFO [2023-08-08 16:05:48] cut tree into: 1 groups
INFO [2023-08-08 16:05:48] -processing spike_tumor_cell_T cells,spike_tumor_cell_T cells_s1
INFO [2023-08-08 16:05:48] define_signif_tumor_subclusters(), tumor: spike_tumor_cell_B cells
INFO [2023-08-08 16:05:48] cut tree into: 1 groups
INFO [2023-08-08 16:05:48] -processing spike_tumor_cell_B cells,spike_tumor_cell_B cells_s1
INFO [2023-08-08 16:05:48] define_signif_tumor_subclusters(), tumor: spike_tumor_cell_Stromal cells
INFO [2023-08-08 16:05:49] cut tree into: 1 groups
INFO [2023-08-08 16:05:49] -processing spike_tumor_cell_Stromal cells,spike_tumor_cell_Stromal cells_s1
INFO [2023-08-08 16:05:49] define_signif_tumor_subclusters(), tumor: spike_tumor_cell_Epithelial cells
INFO [2023-08-08 16:05:49] cut tree into: 1 groups

In [ ]:
# perform infercnv operations to reveal cnv signal
infercnv_obj = infercnv::run(infercnv_obj,
                             cutoff=0.1,  # use 1 for smart-seq, 0.1 for 10x-genomics
                             out_dir="outputs/infercnv/crc/",  # dir is auto-created for storing outputs
                             cluster_by_groups=T,   # cluster
                             denoise=T,
                             HMM=T,
                            analysis_mode='samples',
                                       leiden_resolution=0.01, #initial run returned too many subclusters
                             )

INFO [2023-08-08 18:49:46] ::process_data:Start
INFO [2023-08-08 18:49:46] Checking for saved results.
INFO [2023-08-08 18:49:46] Trying to reload from step 14
INFO [2023-08-08 18:50:18] Trying to reload from step 12
INFO [2023-08-08 18:50:49] Trying to reload from step 11
INFO [2023-08-08 18:51:24] Trying to reload from step 10
INFO [2023-08-08 18:51:59] Trying to reload from step 9
INFO [2023-08-08 18:52:33] Trying to reload from step 8
INFO [2023-08-08 18:53:06] Trying to reload from step 4
INFO [2023-08-08 18:53:24] Using backup from step 4
INFO [2023-08-08 18:53:24] 

	STEP 1: incoming data

INFO [2023-08-08 18:53:24] 

	STEP 02: Removing lowly expressed genes

INFO [2023-08-08 18:53:24] 

	STEP 03: normalization by sequencing depth

INFO [2023-08-08 18:53:24] 

	STEP 04: log transformation of data

INFO [2023-08-08 18:53:24] 

	STEP 08: removing average of reference data (before smoothing)

INFO [2023-08-08 18:53:24] ::subtract_ref_expr_from_obs:Start inv_log=FALSE, use_bounds=TR

In [5]:
infercnv_obj = readRDS("outputs/infercnv/crc/17_HMM_predHMMi6.hmm_mode-samples.infercnv_obj")

In [6]:
# perform infercnv operations to reveal cnv signal
infercnv_obj = infercnv::run(infercnv_obj,
                             cutoff=0.1,  # use 1 for smart-seq, 0.1 for 10x-genomics
                             out_dir="outputs/infercnv/crc/",  # dir is auto-created for storing outputs
                             cluster_by_groups=T,   # cluster
                             denoise=T,
                             HMM=T,
                            analysis_mode='samples',
                                       leiden_resolution=0.01, #initial run returned too many subclusters
                             )

INFO [2023-08-09 15:05:05] ::process_data:Start
INFO [2023-08-09 15:05:05] Checking for saved results.
INFO [2023-08-09 15:05:05] Trying to reload from step 17
INFO [2023-08-09 15:05:23] Using backup HMM from step 17
INFO [2023-08-09 15:05:23] Trying to reload from step 15
INFO [2023-08-09 15:05:54] Using backup from step 15
INFO [2023-08-09 15:05:54] 

	STEP 1: incoming data

INFO [2023-08-09 15:05:54] 

	STEP 02: Removing lowly expressed genes

INFO [2023-08-09 15:05:54] 

	STEP 03: normalization by sequencing depth

INFO [2023-08-09 15:05:54] 

	STEP 04: log transformation of data

INFO [2023-08-09 15:05:54] 

	STEP 08: removing average of reference data (before smoothing)

INFO [2023-08-09 15:05:54] 

	STEP 09: apply max centered expression threshold: 3

INFO [2023-08-09 15:05:54] 

	STEP 10: Smoothing data per cell by chromosome

INFO [2023-08-09 15:05:54] 

	STEP 11: re-centering data across chromosome after smoothing

INFO [2023-08-09 15:05:54] 

	STEP 12: removing average of re

Warning message in parallel::mclapply(seq_along(obj@cell_gene), FUN = par_func, :
“scheduled core 4 did not deliver a result, all values of the job will be affected”


INFO [2023-08-09 19:22:19] Obtaining probabilities post-sampling


ERROR: Error in do.call(rbind, mcmc[[j]]): second argument must be a list
